# 5+0 Blitz Games on lichess.org (September 2019 - January 2020) 

# Step 1) Data Wrangling 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re

%matplotlib inline

In [2]:
# reading in the .pgn file and saving it as a list of lines

list_lines = []
with open('chess_games.pgn', 'r') as f:
    for line in f:
        list_lines.append(line)

In [3]:
# length of the list of lines
len(list_lines)

48384

In [4]:
# total number of games played
event_count = 0
for line in list_lines:
    if '[Event' in line:
        event_count += 1
        
event_count

2306

In [5]:
# preview of the list of lists
list_lines[:25]

['[Event "Rated Blitz game"]\n',
 '[Site "https://lichess.org/5t5Hvluo"]\n',
 '[Date "2020.01.11"]\n',
 '[Round "-"]\n',
 '[White "Oddweird"]\n',
 '[Black "antoshenka13"]\n',
 '[Result "1-0"]\n',
 '[UTCDate "2020.01.11"]\n',
 '[UTCTime "06:04:30"]\n',
 '[WhiteElo "1273"]\n',
 '[BlackElo "1306"]\n',
 '[WhiteRatingDiff "+6"]\n',
 '[BlackRatingDiff "-7"]\n',
 '[Variant "Standard"]\n',
 '[TimeControl "300+0"]\n',
 '[ECO "A41"]\n',
 '[Termination "Time forfeit"]\n',
 '\n',
 '1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b5 6. Bd3 Bg7 7. Nbd2 O-O 8. Ne4 Bb7 9. Nxf6+ Bxf6 10. O-O e5 11. dxe5 dxe5 12. Bxe5 Bxe5 13. Nxe5 Qg5 14. Nf3 Qg4 15. h3 Qh5 16. Qc2 Bxf3 17. gxf3 Qxh3 18. Be4 Ra7 19. f4 c5 20. Bg2 Qh5 21. Rad1 Nc6 22. Bxc6 f5 23. Rd2 g5 24. Rfd1 gxf4 25. exf4 Rg7+ 26. Bg2 Qh3 27. f3 Qxf3 28. Rf1 Qe3+ 29. Rff2 h5 30. b3 h4 31. Re2 Qe6 32. b4 h3 33. bxc5 Rxg2+ 34. Rxg2+ hxg2 35. Rxg2+ 1-0\n',
 '\n',
 '\n',
 '[Event "Rated Blitz game"]\n',
 '[Site "https://lichess.org/a92TptwG"]\n',
 '[Date "

In [6]:
# creating a pandas.Series object for the list of lines
series_lines = pd.Series(list_lines)
len(series_lines)

48384

In [7]:
# cleaning up the newline characters from the series elements
series_lines = series_lines[~(series_lines == '\n')]
series_lines = series_lines.str.replace('\n', '')
len(series_lines)

41466

In [8]:
# preview of the changes made
series_lines.head()

0               [Event "Rated Blitz game"]
1    [Site "https://lichess.org/5t5Hvluo"]
2                      [Date "2020.01.11"]
3                              [Round "-"]
4                       [White "Oddweird"]
dtype: object

In [9]:
list_lines = series_lines.tolist()

# separating each game via a list of lists

list_of_lists = []
intermediate_list = []

for line in list_lines:
    intermediate_list.append(line)
    '''
    each line that doesn't start with [ is the move list for a specific game (1. d4 d6 ..) 
    and is also the last relevant line of data for a given game.
    '''
    if not line.startswith('['):
        list_of_lists.append(intermediate_list)
        intermediate_list = []

In [10]:
# previewing the list of lists; the last game of chess I played on lichess

list_of_lists[0]

['[Event "Rated Blitz game"]',
 '[Site "https://lichess.org/5t5Hvluo"]',
 '[Date "2020.01.11"]',
 '[Round "-"]',
 '[White "Oddweird"]',
 '[Black "antoshenka13"]',
 '[Result "1-0"]',
 '[UTCDate "2020.01.11"]',
 '[UTCTime "06:04:30"]',
 '[WhiteElo "1273"]',
 '[BlackElo "1306"]',
 '[WhiteRatingDiff "+6"]',
 '[BlackRatingDiff "-7"]',
 '[Variant "Standard"]',
 '[TimeControl "300+0"]',
 '[ECO "A41"]',
 '[Termination "Time forfeit"]',
 '1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b5 6. Bd3 Bg7 7. Nbd2 O-O 8. Ne4 Bb7 9. Nxf6+ Bxf6 10. O-O e5 11. dxe5 dxe5 12. Bxe5 Bxe5 13. Nxe5 Qg5 14. Nf3 Qg4 15. h3 Qh5 16. Qc2 Bxf3 17. gxf3 Qxh3 18. Be4 Ra7 19. f4 c5 20. Bg2 Qh5 21. Rad1 Nc6 22. Bxc6 f5 23. Rd2 g5 24. Rfd1 gxf4 25. exf4 Rg7+ 26. Bg2 Qh3 27. f3 Qxf3 28. Rf1 Qe3+ 29. Rff2 h5 30. b3 h4 31. Re2 Qe6 32. b4 h3 33. bxc5 Rxg2+ 34. Rxg2+ hxg2 35. Rxg2+ 1-0']

In [11]:
# creating a pandas.DataFrame for the data and naming the columns
columns = [
    'type_game',
    'site_link',
    'game_date',
    'round_type',
    'player_white',
    'player_black',
    'result',
    'game_start_date',
    'game_start_time',
    'white_elo',
    'black_elo',
    'white_elo_diff',
    'black_elo_diff',
    'variant',
    'time_control',
    'opening',
    'type_termination',
    'moves'
]

df = pd.DataFrame(list_of_lists, columns=columns)
df.head()

,type_game,site_link,game_date,round_type,player_white,player_black,result,game_start_date,game_start_time,white_elo,black_elo,white_elo_diff,black_elo_diff,variant,time_control,opening,type_termination,moves
0,"[Event ""Rated Blitz game""]","[Site ""https://lichess.org/5t5Hvluo""]","[Date ""2020.01.11""]","[Round ""-""]","[White ""Oddweird""]","[Black ""antoshenka13""]","[Result ""1-0""]","[UTCDate ""2020.01.11""]","[UTCTime ""06:04:30""]","[WhiteElo ""1273""]","[BlackElo ""1306""]","[WhiteRatingDiff ""+6""]","[BlackRatingDiff ""-7""]","[Variant ""Standard""]","[TimeControl ""300+0""]","[ECO ""A41""]","[Termination ""Time forfeit""]",1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b...
1,"[Event ""Rated Blitz game""]","[Site ""https://lichess.org/a92TptwG""]","[Date ""2020.01.11""]","[Round ""-""]","[White ""Mohamedalimedoo""]","[Black ""Oddweird""]","[Result ""1-0""]","[UTCDate ""2020.01.11""]","[UTCTime ""05:59:05""]","[WhiteElo ""1200""]","[BlackElo ""1280""]","[WhiteRatingDiff ""+7""]","[BlackRatingDiff ""-7""]","[Variant ""Standard""]","[TimeControl ""300+0""]","[ECO ""A00""]","[Termination ""Normal""]",1. e3 g6 2. d3 Bg7 3. Be2 d6 4. Nf3 Bg4 5. Nc3...
2,"[Event ""Rated Blitz game""]","[Site ""https://lichess.org/oM9lwtc6""]","[Date ""2020.01.11""]","[Round ""-""]","[White ""alislence""]","[Black ""Oddweird""]","[Result ""1/2-1/2""]","[UTCDate ""2020.01.11""]","[UTCTime ""05:48:48""]","[WhiteElo ""1209""]","[BlackElo ""1281""]","[WhiteRatingDiff ""+1""]","[BlackRatingDiff ""-1""]","[Variant ""Standard""]","[TimeControl ""300+0""]","[ECO ""B06""]","[Termination ""Normal""]",1. e4 g6 2. Nf3 d6 3. Bc4 Bg7 4. Ng5 e6 5. Qf3...
3,"[Event ""Rated Blitz game""]","[Site ""https://lichess.org/ueLhV2O9""]","[Date ""2020.01.11""]","[Round ""-""]","[White ""Oddweird""]","[Black ""arpr7878""]","[Result ""1-0""]","[UTCDate ""2020.01.11""]","[UTCTime ""05:32:28""]","[WhiteElo ""1276""]","[BlackElo ""1210""]","[WhiteRatingDiff ""+5""]","[BlackRatingDiff ""-6""]","[Variant ""Standard""]","[TimeControl ""300+0""]","[ECO ""A00""]","[Termination ""Normal""]",1. c3 e5 2. e3 d5 3. Nf3 e4 4. Nd4 Nf6 5. Be2 ...
4,"[Event ""Rated Blitz game""]","[Site ""https://lichess.org/7HHVoqyY""]","[Date ""2020.01.11""]","[Round ""-""]","[White ""Oddweird""]","[Black ""Iulian204""]","[Result ""1-0""]","[UTCDate ""2020.01.11""]","[UTCTime ""05:27:36""]","[WhiteElo ""1270""]","[BlackElo ""1262""]","[WhiteRatingDiff ""+6""]","[BlackRatingDiff ""-6""]","[Variant ""Standard""]","[TimeControl ""300+0""]","[ECO ""D00""]","[Termination ""Normal""]",1. d4 d5 2. Bf4 e6 3. Nf3 Nf6 4. e3 Bd6 5. Bxd...


In [12]:
# dropping columns that I won't be using
df.drop(['site_link', 'round_type', 'game_start_date', 'game_start_time', 'variant', 'type_termination'] ,axis=1, inplace=True)

In [13]:
# cleaning up the square brackets and quotations from the data 
for col in df.columns:
    df[col] = df[col].str.replace('[','').str.replace(']','').str.replace('\n','').str.replace('"','')

In [14]:
# frequency table for the game type
df.type_game.value_counts()

Event Rated Blitz game        1499
Event Rated Rapid game         784
Event Casual Rapid game         12
Event Casual Blitz game          9
Event Casual Standard game       2
Name: type_game, dtype: int64

In [15]:
# filtering out only rated blitz games

df = df[df.type_game == 'Event Rated Blitz game']
df.reset_index(drop=True, inplace=True)

In [16]:
# len should be 1499

len(df)

1499

In [17]:
# dropping the type_game columns
df.drop(['type_game'] ,axis=1, inplace=True)

In [18]:
# cleaning the column values so that they include only neccessary information
df.game_date = df.game_date.str.replace('Date','').str.strip()
df.player_white = df.player_white.str[6:]
df.player_black = df.player_black.str[6:]
df.result = df.result.str.replace('Result','').str.strip()
df.white_elo = df.white_elo.str.replace('WhiteElo','').str.strip()
df.black_elo = df.black_elo.str.replace('BlackElo','').str.strip()
df.white_elo_diff = df.white_elo_diff.str.replace('WhiteRatingDiff','').str.strip()
df.black_elo_diff = df.black_elo_diff.str.replace('BlackRatingDiff','').str.strip()
df.time_control = df.time_control.str.replace('TimeControl','').str.strip()
df.opening = df.opening.str.replace('ECO','').str.strip()
df.moves = df.moves.str.strip()

In [19]:
# previewing the changes, the data is much more readable already
df.head()

,game_date,player_white,player_black,result,white_elo,black_elo,white_elo_diff,black_elo_diff,time_control,opening,moves
0,2020.01.11,Oddweird,antoshenka13,1-0,1273,1306,+6,-7,300+0,A41,1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b...
1,2020.01.11,Mohamedalimedoo,Oddweird,1-0,1200,1280,+7,-7,300+0,A00,1. e3 g6 2. d3 Bg7 3. Be2 d6 4. Nf3 Bg4 5. Nc3...
2,2020.01.11,alislence,Oddweird,1/2-1/2,1209,1281,+1,-1,300+0,B06,1. e4 g6 2. Nf3 d6 3. Bc4 Bg7 4. Ng5 e6 5. Qf3...
3,2020.01.11,Oddweird,arpr7878,1-0,1276,1210,+5,-6,300+0,A00,1. c3 e5 2. e3 d5 3. Nf3 e4 4. Nd4 Nf6 5. Be2 ...
4,2020.01.11,Oddweird,Iulian204,1-0,1270,1262,+6,-6,300+0,D00,1. d4 d5 2. Bf4 e6 3. Nf3 Nf6 4. e3 Bd6 5. Bxd...


In [20]:
df.time_control.value_counts()

300+0    1499
Name: time_control, dtype: int64

In [21]:
# Since the only blitz games I've played are 5+0 I'll drop this column, I also don't need white_elo_diff or black_elo_diff
# as I can create calculated fields later on in Tableau if I need to

df.drop(['time_control', 'white_elo_diff', 'black_elo_diff'], axis=1, inplace=True)

In [22]:
df.head()

,game_date,player_white,player_black,result,white_elo,black_elo,opening,moves
0,2020.01.11,Oddweird,antoshenka13,1-0,1273,1306,A41,1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b...
1,2020.01.11,Mohamedalimedoo,Oddweird,1-0,1200,1280,A00,1. e3 g6 2. d3 Bg7 3. Be2 d6 4. Nf3 Bg4 5. Nc3...
2,2020.01.11,alislence,Oddweird,1/2-1/2,1209,1281,B06,1. e4 g6 2. Nf3 d6 3. Bc4 Bg7 4. Ng5 e6 5. Qf3...
3,2020.01.11,Oddweird,arpr7878,1-0,1276,1210,A00,1. c3 e5 2. e3 d5 3. Nf3 e4 4. Nd4 Nf6 5. Be2 ...
4,2020.01.11,Oddweird,Iulian204,1-0,1270,1262,D00,1. d4 d5 2. Bf4 e6 3. Nf3 Nf6 4. e3 Bd6 5. Bxd...


In [23]:
# making sure that the result column only includes win, loss or draw values
df.result.value_counts()

0-1        725
1-0        709
1/2-1/2     65
Name: result, dtype: int64

In [24]:
# setting which color I played
df['played_as'] = df.player_white.apply(lambda x: 'White' if x == 'Oddweird' else 'Black')

# setting my current elo for each game at the time the game was played
df['elo'] = np.nan
df.loc[df['played_as'] == 'White', 'elo'] = df.white_elo
df.loc[df['played_as'] == 'Black', 'elo'] = df.black_elo

# setting my opponent for each game
df['opponent'] = np.nan
df.loc[df['played_as'] == 'White', 'opponent'] = df.player_black
df.loc[df['played_as'] == 'Black', 'opponent'] = df.player_white

# setting my opponent's elo for each game at the time the game was played
df['opponent_elo'] = np.nan
df.loc[df['played_as'] == 'White', 'opponent_elo'] = df.black_elo
df.loc[df['played_as'] == 'Black', 'opponent_elo'] = df.white_elo

# dropping columns which are no longer needed
df.drop(['player_white', 'player_black', 'white_elo', 'black_elo'], axis=1, inplace=True)

In [25]:
df.head()

,game_date,result,opening,moves,played_as,elo,opponent,opponent_elo
0,2020.01.11,1-0,A41,1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b...,White,1273,antoshenka13,1306
1,2020.01.11,1-0,A00,1. e3 g6 2. d3 Bg7 3. Be2 d6 4. Nf3 Bg4 5. Nc3...,Black,1280,Mohamedalimedoo,1200
2,2020.01.11,1/2-1/2,B06,1. e4 g6 2. Nf3 d6 3. Bc4 Bg7 4. Ng5 e6 5. Qf3...,Black,1281,alislence,1209
3,2020.01.11,1-0,A00,1. c3 e5 2. e3 d5 3. Nf3 e4 4. Nd4 Nf6 5. Be2 ...,White,1276,arpr7878,1210
4,2020.01.11,1-0,D00,1. d4 d5 2. Bf4 e6 3. Nf3 Nf6 4. e3 Bd6 5. Bxd...,White,1270,Iulian204,1262


In [26]:
# defining a new column for game results
df['new_result'] = np.nan

In [27]:
# function to determine win/loss/draw for games I played as white 
# based on the current value in the result column
def white_result(s):
    if s == '1-0':
        return 'Win'
    if s == '0-1':
        return 'Loss'
    return 'Draw'

# applying the function to the result column for games where I played white 
# and setting new values in the new_result column
df.loc[df.played_as == 'White', 'new_result'] = df.loc[df.played_as == 'White', 'result'].apply(white_result)

In [28]:
# function to determine win/loss/draw for games I played as black 
# based on the current value in the result column
def black_result(s):
    if s == '1-0':
        return 'Loss'
    if s == '0-1':
        return 'Win'
    return 'Draw'

# applying the function to the result column for games where I played black 
# and setting new values in the new_result column
df.loc[df.played_as == 'Black', 'new_result'] = df.loc[df.played_as == 'Black', 'result'].apply(black_result)

In [29]:
# dropping the original result column which I don't need anymore and renaming the new_result column to result
df.drop(['result'], axis=1, inplace=True)
df.rename({'new_result': 'result'}, axis=1, inplace=True)

# re-ordering the remaining columns for intuitive readability
col_order = [
    'game_date',
    'played_as',
    'elo',
    'opponent',
    'opponent_elo',
    'opening',
    'result',
    'moves'
]

df = df[col_order]

In [30]:
# preving changes to the result column
df.head()

,game_date,played_as,elo,opponent,opponent_elo,opening,result,moves
0,2020.01.11,White,1273,antoshenka13,1306,A41,Win,1. d4 d6 2. Bf4 Nf6 3. Nf3 g6 4. e3 a6 5. c3 b...
1,2020.01.11,Black,1280,Mohamedalimedoo,1200,A00,Loss,1. e3 g6 2. d3 Bg7 3. Be2 d6 4. Nf3 Bg4 5. Nc3...
2,2020.01.11,Black,1281,alislence,1209,B06,Draw,1. e4 g6 2. Nf3 d6 3. Bc4 Bg7 4. Ng5 e6 5. Qf3...
3,2020.01.11,White,1276,arpr7878,1210,A00,Win,1. c3 e5 2. e3 d5 3. Nf3 e4 4. Nd4 Nf6 5. Be2 ...
4,2020.01.11,White,1270,Iulian204,1262,D00,Win,1. d4 d5 2. Bf4 e6 3. Nf3 Nf6 4. e3 Bd6 5. Bxd...


In [31]:
# converting the date values in the game_date column from YYYY.mm.dd to YYYY-mm-dd 
# using dt.datetime.strptime and strftime from the datetime library
df['game_date'] = df.game_date.apply(lambda x: dt.datetime.strptime(x,'%Y.%m.%d').strftime('%Y-%m-%d'))

In [32]:
# converting the values in the elo and opponent_elo column to integers for aggregations
df['elo'] = df.elo.astype(int)
df['opponent_elo'] = df.opponent_elo.astype(int)

In [33]:
# reversing the rows of the DataFrame and resetting the index so that plots using
# time series will be more intuitive
df = df.iloc[::-1].reset_index(drop=True)

In [34]:
# previwing the changes
df.head()

,game_date,played_as,elo,opponent,opponent_elo,opening,result,moves
0,2019-09-01,Black,1500,treap,1394,A00,Loss,1. g3 d5 2. Bg2 Nf6 3. Nf3 c5 4. d3 b5 5. O-O ...
1,2019-09-01,Black,1264,DARIORITUM,1265,C51,Loss,1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. b4 Bb6 5. O-...
2,2019-09-01,White,1144,sameh2pho,1127,A01,Win,1. b3 e5 2. Bb2 d6 3. e4 Nf6 4. Bb5+ c6 5. Bc4...
3,2019-09-02,Black,1228,Etaash,1207,A40,Loss,1. d4 c6 2. Bf4 d5 3. e3 Bf5 4. Nf3 e6 5. Bd3 ...
4,2019-09-02,Black,1151,yevbar,1227,C01,Loss,1. e4 e6 2. d4 d5 3. exd5 exd5 4. Nc3 Nf6 5. N...


In [35]:
example = df.moves[0]
example

'1. g3 d5 2. Bg2 Nf6 3. Nf3 c5 4. d3 b5 5. O-O e6 6. Bg5 h6 7. Bxf6 Qxf6 8. Nbd2 c4 9. c3 cxd3 10. exd3 b4 11. Qa4+ Nd7 12. Qc6 Rb8 13. d4 Bb7 14. Qa4 Be7 15. Ne5 Rd8 16. Ndf3 Bd6 17. Nxd7 Rxd7 18. Ne5 Bxe5 19. dxe5 Qxe5 20. Qxb4 Qd6 21. Qxd6 Rxd6 22. c4 d4 23. Bxb7 a5 24. Rfd1 e5 25. Rac1 f6 26. c5 Rd5 27. Bxd5 Ke7 28. c6 Rd8 29. Bc4 Kd6 30. Ba6 e4 31. c7 Rc8 32. Bxc8 Kd5 33. Ba6 e3 34. c8=Q f5 35. Qc5+ Ke4 36. Bb7# 1-0'

In [36]:
# using a regex to split up the moves list based on the \d. before each move (1. 2. 3. .. 46.)
move_list = re.split(r'\d+\.', example)
move_list

['',
 ' g3 d5 ',
 ' Bg2 Nf6 ',
 ' Nf3 c5 ',
 ' d3 b5 ',
 ' O-O e6 ',
 ' Bg5 h6 ',
 ' Bxf6 Qxf6 ',
 ' Nbd2 c4 ',
 ' c3 cxd3 ',
 ' exd3 b4 ',
 ' Qa4+ Nd7 ',
 ' Qc6 Rb8 ',
 ' d4 Bb7 ',
 ' Qa4 Be7 ',
 ' Ne5 Rd8 ',
 ' Ndf3 Bd6 ',
 ' Nxd7 Rxd7 ',
 ' Ne5 Bxe5 ',
 ' dxe5 Qxe5 ',
 ' Qxb4 Qd6 ',
 ' Qxd6 Rxd6 ',
 ' c4 d4 ',
 ' Bxb7 a5 ',
 ' Rfd1 e5 ',
 ' Rac1 f6 ',
 ' c5 Rd5 ',
 ' Bxd5 Ke7 ',
 ' c6 Rd8 ',
 ' Bc4 Kd6 ',
 ' Ba6 e4 ',
 ' c7 Rc8 ',
 ' Bxc8 Kd5 ',
 ' Ba6 e3 ',
 ' c8=Q f5 ',
 ' Qc5+ Ke4 ',
 ' Bb7# 1-0']

In [37]:
# this method adds 1 extra 'move' in the form of a blank element at the beginning of the list
len(move_list)

37

In [38]:
def count_moves(s):
    move_list = re.split(r'\d+\.', s)
    new_move_list = []
    i = 1
    
    for move in move_list:
        # skipping the first blank element in the list of moves
        if move == '':
            continue
        new_move_list.append(str(i) + '. ' + move.strip())
        i += 1
    return len(new_move_list)

df['num_moves'] = df.moves.apply(count_moves)

In [39]:
df.head()

,game_date,played_as,elo,opponent,opponent_elo,opening,result,moves,num_moves
0,2019-09-01,Black,1500,treap,1394,A00,Loss,1. g3 d5 2. Bg2 Nf6 3. Nf3 c5 4. d3 b5 5. O-O ...,36
1,2019-09-01,Black,1264,DARIORITUM,1265,C51,Loss,1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. b4 Bb6 5. O-...,35
2,2019-09-01,White,1144,sameh2pho,1127,A01,Win,1. b3 e5 2. Bb2 d6 3. e4 Nf6 4. Bb5+ c6 5. Bc4...,19
3,2019-09-02,Black,1228,Etaash,1207,A40,Loss,1. d4 c6 2. Bf4 d5 3. e3 Bf5 4. Nf3 e6 5. Bd3 ...,31
4,2019-09-02,Black,1151,yevbar,1227,C01,Loss,1. e4 e6 2. d4 d5 3. exd5 exd5 4. Nc3 Nf6 5. N...,26


In [40]:
# re-ordering the columns one last time for the final DataFrame
col_order = [
    'game_date',
    'played_as',
    'elo',
    'opponent',
    'opponent_elo',
    'opening',
    'num_moves',
    'result',
    'moves'
]

df = df[col_order]

# renaming the columns for exporting to Excel/Tableau
map = {
    'game_date': 'Date',
    'played_as': 'ColorPieces',
    'elo': 'ELO',
    'opponent': 'Opponent',
    'opponent_elo': 'OpponentELO',
    'opening': 'Opening',
    'num_moves': 'NumMoves',
    'result': 'Result',
    'moves': 'PGN'
}

df.rename(map, axis=1, inplace=True)

In [41]:
df.head()

,Date,ColorPieces,ELO,Opponent,OpponentELO,Opening,NumMoves,Result,PGN
0,2019-09-01,Black,1500,treap,1394,A00,36,Loss,1. g3 d5 2. Bg2 Nf6 3. Nf3 c5 4. d3 b5 5. O-O ...
1,2019-09-01,Black,1264,DARIORITUM,1265,C51,35,Loss,1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. b4 Bb6 5. O-...
2,2019-09-01,White,1144,sameh2pho,1127,A01,19,Win,1. b3 e5 2. Bb2 d6 3. e4 Nf6 4. Bb5+ c6 5. Bc4...
3,2019-09-02,Black,1228,Etaash,1207,A40,31,Loss,1. d4 c6 2. Bf4 d5 3. e3 Bf5 4. Nf3 e6 5. Bd3 ...
4,2019-09-02,Black,1151,yevbar,1227,C01,26,Loss,1. e4 e6 2. d4 d5 3. exd5 exd5 4. Nc3 Nf6 5. N...


In [42]:
# keeping all columns but the PGN column
df = df.loc[:,'Date':'Result']

In [43]:
# creating a unique identified column
df['Id'] = range(1, len(df)+1)

In [44]:
# re-ordering columns 
col_order = [
    'Id',
    'Date',
    'ColorPieces',
    'ELO',
    'Opponent',
    'OpponentELO',
    'Opening',
    'NumMoves',
    'Result'
]

df = df[col_order]

In [45]:
# creating a second DataFrame called elo_data which will store my End-of-Day ELO for every day that 
# Using this I can easily visualize an accurate time series of ELO over time without having to do much work in Tableau
elo_data = df.drop_duplicates(keep='last', subset='Date')[['Date', 'ELO']].set_index('Date')

# Step 2) Exporting the Data

In [46]:
# the first DataFrame to be exported to Excel (sheel 1)
elo_data.head()

,ELO
Date,
2019-09-01,1144
2019-09-02,977
2019-09-03,993
2019-09-04,969
2019-09-05,985


In [47]:
# the second DataFrame to be exported to Excel (sheet 2)
df.head()

,Id,Date,ColorPieces,ELO,Opponent,OpponentELO,Opening,NumMoves,Result
0,1,2019-09-01,Black,1500,treap,1394,A00,36,Loss
1,2,2019-09-01,Black,1264,DARIORITUM,1265,C51,35,Loss
2,3,2019-09-01,White,1144,sameh2pho,1127,A01,19,Win
3,4,2019-09-02,Black,1228,Etaash,1207,A40,31,Loss
4,5,2019-09-02,Black,1151,yevbar,1227,C01,26,Loss


In [48]:
# exporting DataFrames to excel
with pd.ExcelWriter('chess_games.xlsx') as writer:
    elo_data.to_excel(writer, sheet_name='elo_data')
    df.to_excel(writer, sheet_name='game_data', index=False)

# Step 3) Using Tableau for visualizations and insights

In [58]:
%%html
<img src="chessELO.png">
<img src="chessOPENINGS.png">

# Thanks for reading! 